<a href="https://colab.research.google.com/github/rmferrer/cnc/blob/main/src/cnc_manufacturability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CNC Machine Manufacturability**

---



# Imports

In [7]:
!pip install trimesh scipy pyglet

import numpy as np
import trimesh

from typing import Iterable, Union
from pathlib import Path

import os
import ipywidgets as widgets
from IPython.display import Image, display

# # attach to logger so trimesh messages will be printed to console
# trimesh.util.attach_to_log()

!git clone https://github.com/rmferrer/cnc.git

REPO_DIR = "/content/cnc"
ASSET_DIR = f"{REPO_DIR}/assets"
IMG_DIR = f"{ASSET_DIR}/img"
STL_DIR = f"{ASSET_DIR}/stls"


fatal: destination path 'cnc' already exists and is not an empty directory.


# Utils

In [3]:
ToolAxis = Union[np.ndarray, Iterable[float]]

class CADAnalysisError(Exception):
    """Raised when the CAD file cannot be analysed."""

def load_mesh(file_path: str) -> trimesh.Trimesh:
    """Load a CAD mesh from *file_path*.

    Parameters
    ----------
    file_path:
        Path to a CAD file supported by :mod:`trimesh` (e.g. ``.stl``).

    Returns
    -------
    trimesh.Trimesh
        The triangulated mesh extracted from the file.

    Raises
    ------
    FileNotFoundError
        If the file does not exist.
    CADAnalysisError
        If the file cannot be converted into a mesh.
    """

    path = Path(file_path)
    if not path.exists():
        raise FileNotFoundError(path)

    mesh = trimesh.load(path, force="mesh")

    if isinstance(mesh, trimesh.Scene):
        geometries = [g for g in mesh.geometry.values() if isinstance(g, trimesh.Trimesh)]
        if not geometries:
            raise CADAnalysisError("CAD file does not contain a solid mesh")
        mesh = trimesh.util.concatenate(geometries)

    if not isinstance(mesh, trimesh.Trimesh) or mesh.is_empty:
        raise CADAnalysisError("CAD file could not be interpreted as a solid mesh")

    if not mesh.is_watertight:
        mesh = mesh.fill_holes()

    return mesh


# Manufacturability Logic

In [ ]:
def has_undercut(mesh: trimesh.Trimesh, tool_axis: np.ndarray, tolerance: float) -> bool:
    """Return ``True`` when *mesh* contains surfaces that create an undercut."""

    face_normals = mesh.face_normals
    face_centroids = mesh.triangles_center

    projections = face_normals @ tool_axis
    min_plane = mesh.bounds[0, 2]

    is_downward_facing = projections < -tolerance
    away_from_base = (face_centroids[:, 2] - min_plane) > tolerance

    return bool(np.any(is_downward_facing & away_from_base))

def is_3_axis_cnc_manufacturable(mesh: trimesh.Trimesh, tool_axis: np.ndarray, tolerance: float) -> bool:
  return not has_undercut(mesh, tool_axis, tolerance)

# Visualization

In [39]:
# @title STL picker & viewer {"display-mode": "form"}

import ipywidgets as widgets
from IPython.display import display, HTML
import numpy as np
import trimesh
from trimesh.viewer.notebook import scene_to_notebook

# 1) Build the dropdown options from the STL folder
files_in_stl_raw = !ls /content/cnc/assets/stls
stl_filenames = [
    f
    for line in files_in_stl_raw
    for f in line.split()
    if f.endswith(".stl")
]

file_dropdown = widgets.Dropdown(
    options=stl_filenames,
    description="STL:",
    layout=widgets.Layout(width="60%")
)

# Area where we'll show the status + mesh
output = widgets.Output()

def update_view(change):
    if change.get("name") != "value":
        return

    stl_filename = change["new"]
    stl_path = f"{STL_DIR}/{stl_filename}"

    with output:
        output.clear_output()

        form = load_mesh(stl_path)
        is_ok = is_3_axis_cnc_manufacturable(
            form, np.array([0, 0, 1]), 0.01
        )

        status_text = (
            "✅ 3-axis CNC manufacturable"
            if is_ok
            else "❌ NOT 3-axis CNC manufacturable"
        )
        color = "green" if is_ok else "red"

        # Prominent text
        display(HTML(f"""
        <div style="
            font-size: 28px;
            font-weight: 700;
            color: {color};
            margin: 16px 0;
        ">
          {status_text}
        </div>
        """))

        # Build a Scene and show it inline in the notebook
        if isinstance(form, trimesh.Scene):
            scene = form
        else:
            scene = trimesh.Scene(form)

        display(scene_to_notebook(scene, height=400))

# 2) Wire dropdown -> callback
file_dropdown.observe(update_view, names="value")

# 3) Display UI
display(widgets.VBox([file_dropdown, output]))

# 4) Run once for initial selection
update_view({"name": "value", "new": file_dropdown.value})